# Robot stuff

In this notebook we can check how to sample the trajectory of the robot, how to get velocity profiles from the trajectory, which can be used to control the robot, etc.  

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

In [ ]:
from trajectory import Trajectory
from environment import Environment

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_complexity = 3 #number of robot sample positions
dim = 2 # dimension of setup. 

trajectory = Trajectory(n_complexity, dim=dim, model='polynomial')
trajectory.set_coeffs(seed=2)
environment = Environment(n_anchors)
environment.set_random_anchors(seed=2)

n_positions = 5
times = trajectory.get_times(n_samples=n_positions)
basis = trajectory.get_basis(times=times)
sample_points = trajectory.get_sampling_points(basis=basis)

plt.figure()
environment.plot()
trajectory.plot(basis, color='orange', label='ground truth')
plt.legend()

## Compute the curvature at each point. 

For now there is still a problem. Since the points are not resampled in time, there is not only a radial acceleration (the one that makes the robot turn) but also a tangential radiation. In plot below, the orange lines showing the curvature radius, are therefore oriented in the direction of the curve.

For now I am not quite sure if this is a problem, or if we can still try to incorporate this in the controls for the robot. 

In [ ]:
#curvature
trajectory.plot(basis, color='orange', label='ground truth')
plt.axis('equal')

# compute derivatives of the basis vectors.
basis_prime = trajectory.get_basis_prime(times=times)
basis_twoprime = trajectory.get_basis_twoprime(times=times)

# see wikipedia for these definitions: 
# https://en.wikipedia.org/wiki/Curvature 
# Section: Precise definition
tangents = trajectory.coeffs.dot(basis_prime)
unit_tangents = tangents / np.linalg.norm(tangents, axis=0)
curvatures = trajectory.coeffs.dot(basis_twoprime) 
normal_vectors = curvatures / np.linalg.norm(curvatures, axis=0)

# extract only radial component of acceleration 
radial_acc = []
for norm, unit in zip(normal_vectors.T, unit_tangents.T):
    norm = np.r_[norm.reshape((-1, )), 0]
    unit = np.r_[unit.reshape((-1, )), 0]
    radial = np.cross(unit, np.cross(norm, unit))
    radial_acc.append(radial[:2])
radial_acc = np.array(radial_acc).T
radii = 1 / np.linalg.norm(radial_acc, axis=0)
radial_norm = radial_acc * radii
print(radii)

from matplotlib.patches import Circle

for i in range(sample_points.shape[1]):
    s = sample_points[:, i]
    t = s + unit_tangents[:, i]
    n = s + normal_vectors[:, i]
    r = s + radial_acc[:, i]
    plt.plot([s[0], t[0]], [s[1], t[1]], ':', color='black')
    plt.plot([s[0], n[0]], [s[1], n[1]], ':', color='orange')
    plt.plot([s[0], r[0]], [s[1], r[1]], ':', color='blue')
    
    
    radius = radii[i]
    center = s + radius * radial_norm[:, i]
    circ = Circle(xy=center, radius=radius, alpha=0.1, color='blue')
    plt.scatter(*center, color='blue', marker='+')
    plt.gca().add_artist(circ)

In [ ]:
from scipy.signal import resample

plt.plot(times, sample_points[0, :], label='x', marker='+')
plt.plot(times, sample_points[1, :], label='y', marker='+')
plt.title('trajectory over time.')
plt.xlabel('time')
plt.legend()